In [1]:
import pandas as pd
import ipywidgets as widgets
from collections import Counter

In [2]:
predictions = pd.read_csv('CSV/predictions/predictions.csv')
predictions.sort_values(by=['predicted_point_range', 'prob_3', 'prob_2', 'prob_1', 'prob_0'], ascending=False, inplace=True)
season = pd.read_csv('CSV/raw_gameweek_data/gameweeks-2020-21.csv').rename(columns={'team':'plays_for'})
all_players = season[season['round'] == max(season['round'])][['name', 'value', 'position', 'plays_for']].drop_duplicates(subset=['name'])

In [3]:
predictions.sort_values(by=['predicted_point_range', 'prob_3', 'prob_2', 'prob_1', 'prob_0'], ascending=False)[:20]

,name,value,position,plays_for,shift_opponent,predicted_point_range,prob_0,prob_1,prob_2,prob_3
60,Timo Werner,92,FWD,Chelsea,Southampton,3.0,0.007761,0.004832,0.117724,0.869683
182,Bruno Miguel Borges Fernandes,115,MID,Man United,Newcastle,3.0,0.009692,0.071042,0.234171,0.685095
213,Jamie Vardy,100,FWD,Leicester,Aston Villa,3.0,0.140705,0.009322,0.206703,0.643271
20,Ademola Lookman,50,MID,Fulham,Sheffield United,2.0,0.005145,0.020938,0.523651,0.450265
108,Billy Sharp,55,FWD,Sheffield United,Fulham,2.0,0.058029,0.061017,0.488707,0.392247
107,Olivier Giroud,67,FWD,Chelsea,Southampton,2.0,0.016414,0.157034,0.438644,0.387909
262,Alexis Mac Allister,53,MID,Brighton,Crystal Palace,2.0,0.037298,0.013964,0.649141,0.299597
362,Kai Havertz,82,MID,Chelsea,Southampton,2.0,0.144627,0.065638,0.499106,0.290629
176,Anthony Martial,87,FWD,Man United,Newcastle,2.0,0.055335,0.181316,0.505430,0.257919
84,Marcos Alonso,57,DEF,Chelsea,Southampton,2.0,0.188527,0.225639,0.347643,0.238191


In [4]:
your_squad = []

def add_player(b):
    player_name = players.value
    if player_name in tuple(all_players['name']):
        if player_name in your_squad:
            print('Player already in squad')
        else:
            your_squad.append(player_name)
    else:
        raise ValueError('Player name not valid')
    
def remove_player(b):
    player_name = squad.value
    if player_name in your_squad:
        your_squad.remove(player_name)
    else:
        raise ValueError('Player name not valid')
    
def restart_list(b):
    your_squad.clear()
    
def view_full_squad(b):
    if not your_squad:
        raise ValueError('Squad list is empty.')
    try:
        player_rows = [predictions[predictions['name'] == name] for name in your_squad]
        squad_predictions = pd.concat(player_rows)
        squad_predictions['position'] = pd.Categorical(squad_predictions['position'], categories=['GK', 'DEF', 'MID', 'FWD'])
        squad_predictions.sort_values(by=['position', 'plays_for', 'name'], inplace=True)
    except:
        squad_predictions = pd.DataFrame()
    try:
        player_rows_2 = [all_players[all_players['name'] == name] for name in your_squad if name not in predictions['name'].unique()]
        squad_predictions_null = pd.concat(player_rows_2)
        squad_predictions_null['position'] = pd.Categorical(squad_predictions_null['position'], categories=['GK', 'DEF', 'MID', 'FWD'])
        squad_predictions_null.sort_values(by=['position', 'plays_for', 'name'], inplace=True)
    except:
        squad_predictions_null = pd.DataFrame()
    teams_in_squad = Counter(pd.concat([squad_predictions, squad_predictions_null])['plays_for'])
    if max(teams_in_squad.values()) > 3:
        raise ValueError('Invalid team distribution in squad list. Must have 3 or less players from the same team.')
    display(squad_predictions.style.set_caption('Likely to play'))
    display(squad_predictions_null.style.set_caption('Unlikely to play'))
    
    
def make_transfer(b):
    budget = remaining_budget.value
    if not your_squad:
        raise ValueError('Squad list is empty.')
    player_rows = [predictions[predictions['name'] == name] for name in your_squad] + [all_players[all_players['name'] == name] for name in your_squad if name not in predictions['name'].unique()]
    squad = pd.concat(player_rows).sort_values(by=['predicted_point_range', 'prob_3', 'prob_2', 'prob_1', 'prob_0', 'value'], ascending=[True, True, True, True, True, False], na_position='first')
    teams_in_squad = Counter(tuple(squad['plays_for']))
    if max(teams_in_squad.values()) > 3:
        raise ValueError('Invalid team distribution in squad list. Must have 3 or less players from the same team.')
    statement = True
    i = 0
    while statement:
        best_replacement = predictions.iloc[i]
        if best_replacement['name'] in tuple(squad['name']):
            i += 1
            continue
        budget_needed = best_replacement['value'] - budget
        players_to_replace = squad[(squad['value'] >= budget_needed) & (squad['position'] == best_replacement['position'])]
        if not tuple(players_to_replace.index):
            i += 1
            continue
        for j, player in players_to_replace.iterrows():
            teams_in_squad[player['plays_for']] = teams_in_squad[player['plays_for']] - 1
            teams_in_squad[best_replacement['plays_for']] = teams_in_squad[best_replacement['plays_for']] + 1
            if max(teams_in_squad.values()) > 3:
                continue
            elif pd.isnull(player['predicted_point_range']):
                display(pd.DataFrame(player).T.style.set_caption('Replace'))
                display(pd.DataFrame(best_replacement).T.style.set_caption('With'))
                statement = False
                break
            elif (best_replacement['predicted_point_range'] >= player['predicted_point_range']) & (best_replacement['prob_{}'.format(int(best_replacement['predicted_point_range']))] > player['prob_{}'.format(int(best_replacement['predicted_point_range']))]):
                display(pd.DataFrame(player).T.style.set_caption('Replace'))
                display(pd.DataFrame(best_replacement).T.style.set_caption('With'))
                statement = False
                break
            else:
                continue
        i += 1
        continue

In [5]:
players = widgets.Combobox(
    placeholder='Choose Player',
    options=tuple(all_players['name'].sort_values()),
    description='Name:',
    ensure_option=True,
    disabled=False)

add_button = widgets.Button(
    description='Click to add player to your squad.',
    disabled=False,
    button_style='success',
    layout=widgets.Layout(width='50%'))

add_button.on_click(add_player)

display(players), display(add_button)
print()

Combobox(value='', description='Name:', ensure_option=True, options=('Aaron Connolly', 'Aaron Cresswell', 'Aar…

Button(button_style='success', description='Click to add player to your squad.', layout=Layout(width='50%'), s…

In [6]:
# if you would like to remove a player from your squad, you will first need to re-run this cell before and after
# to refresh your squad list in the text list.
squad = widgets.Combobox(
    placeholder='Choose Player',
    options=tuple(sorted(your_squad)),
    description='Name:',
    ensure_option=True,
    disabled=False)

remove_button = widgets.Button(
    description='Click to remove player from your squad.',
    disabled=False,
    button_style='warning',
    layout=widgets.Layout(width='50%'))

remove_button.on_click(remove_player)

display(squad), display(remove_button)
print()

Combobox(value='', description='Name:', ensure_option=True, placeholder='Choose Player')

Button(button_style='warning', description='Click to remove player from your squad.', layout=Layout(width='50%…

In [7]:
restart_button = widgets.Button(
    description='Click to reset squad list.',
    disabled=False,
    button_style='danger',
    layout=widgets.Layout(width='50%'))

restart_button.on_click(restart_list)

display(restart_button)
print()

Button(button_style='danger', description='Click to reset squad list.', layout=Layout(width='50%'), style=Butt…

In [8]:
view_full_squad_button = widgets.Button(
    description='Click to view your squad.',
    disabled=False,
    button_style='info',
    layout=widgets.Layout(width='50%'))

view_full_squad_button.on_click(view_full_squad)

display(view_full_squad_button)
print()

Button(button_style='info', description='Click to view your squad.', layout=Layout(width='50%'), style=ButtonS…

In [9]:
style = {'description_width': 'initial'}

remaining_budget = widgets.IntText(
    value=None,
    description='Remaining Budget:',
    disabled=False,
    style=style)

make_transfer_button = widgets.Button(
    description='Click to find best transfer.',
    disabled=False,
    button_style='info',
    layout=widgets.Layout(width='50%'))

make_transfer_button.on_click(make_transfer)

display(remaining_budget), display(make_transfer_button)
print()

IntText(value=0, description='Remaining Budget:', style=DescriptionStyle(description_width='initial'))

Button(button_style='info', description='Click to find best transfer.', layout=Layout(width='50%'), style=Butt…